The article "prediction of Wine Quality Using Machine Learning" states that it used the red wine dataset. Paragraph 2.1 mentions that the dataset contains 4898 entries. When examing the datasets, this number corresponds to the white wine dataset. This confusion in consistent throughout the article. From this point on the white wine dataset will be used, eventhough the article states that the red wine dataset is used.

In [ ]:
# used libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from shap import initjs

print(tf.__version__)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Input
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from scipy.stats import pearsonr
from keras.optimizers import Adam

In [ ]:
#import the winequality dataset

# Define the relative path to the CSV file
relative_path = 'wine+quality/winequality-white.csv'
# Read the CSV file using pandas
wine_df = pd.read_csv(relative_path, delimiter=';')

features = wine_df.drop('quality', axis=1)
target = wine_df['quality']

len(features)


Feature scaling was applied using standardization. Following code standardizes the dataset.

In [ ]:
scaler = StandardScaler()

features_scaled = scaler.fit_transform(features)

feature_scaled_df = pd.DataFrame(features_scaled, columns=features.columns)
feature_scaled_df

Create the train, validation and test set from the original data set

In [ ]:
# Step 1: Split the data into training and temporary sets
# The training set will contain 60% of the data, and the temporary set will contain 40% of the data
features_train, features_temp, target_train, target_temp = train_test_split(features_scaled, target, test_size=0.4, random_state=1)

# Step 1: Split the data into training and temporary sets
# The training set will contain 60% of the data, and the temporary set will contain 40% of the data
features_test, features_validate, target_test, target_validate = train_test_split(features_temp, target_temp, test_size=0.5, random_state=1)

# Now, features_train and target_train contain the training data (60% of the original data)
# features_test and target_test contain the testing data (20% of the original data)
# features_validate and target_validate contain the validation data (20% of the original data)


Create a ANN model with:
- 1 input layer with 11 neurons
- 3 hidden layers with 15 neurons
- 1 output layer with 1 neuron

In [ ]:
# Initialize the Sequential model
model = tf.keras.Sequential()
# model = Sequential()

#add layers
model.add(Input(shape=(11,))) # input layer
model.add(Dense(15, activation='relu')) # first hidden layer
model.add(Dense(15, activation='relu')) # second hidden layer
model.add(Dense(15, activation='relu')) # third hidden layer
model.add(Dense(1, activation='linear')) # Output layer

# Compile the model with mean squared error loss and Adam optimizer
# Also include mean squared error, mean absolute error, and mean absolute percentage error as metrics
model.compile(loss='mse', optimizer='adam', metrics=['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error'])

# Print the summary of the model architecture
model.summary()

In [ ]:
# Train the model with the training data
# - epochs: Number of times the model will iterate over the entire training dataset
# - batch_size: Number of samples per gradient update
# - validation_data: Data on which to evaluate the loss and metrics at the end of each epoch
# - verbose: Verbosity mode (1 = progress bar)
history = model.fit(features_train, target_train, epochs=50, batch_size=32, validation_data=(features_validate, target_validate), verbose=1)


In [ ]:
# Evaluate the model on the test data
test_loss = model.evaluate(features_test, target_test, verbose=1)

# Print the test loss
print("Test loss:", test_loss)

# Make predictions on the test data
predictions = model.predict(features_test)



In [ ]:
# Function to calculate metrics (R, MSE, MAPE)
def calculate_metrics(y_true, y_pred):
    # Calculate R (Pearson correlation coefficient)
    r, _ = pearsonr(y_true, y_pred)

    # Calculate MSE
    mse = mean_squared_error(y_true, y_pred)

    # Calculate MAPE
    mape = mean_absolute_percentage_error(y_true, y_pred)

    return r, mse, mape

In [ ]:
# Make predictions on the training and test data
train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

In [ ]:
# Convert to 1D numpy array
train_predictions = train_predictions.flatten()
test_predictions = test_predictions.flatten()

In [ ]:
# Calculate metrics for training data set
train_r, train_mse, train_mape = calculate_metrics(target_train, train_predictions)

# # Calculate metrics for testing data set
test_r, test_mse, test_mape = calculate_metrics(target_test, test_predictions)

In [ ]:
import shap

background_data = features_train  # Use a subset of the training data for background
test_data = features_test[:5]

explainer = shap.KernelExplainer(model, background_data)
shap_values = explainer.shap_values(test_data)
fig, ax = plt.subplots(4, 3, figsize=(20, 15))
ax = ax.flatten()

for i in range(11):
    shap.dependence_plot(i, shap_values, test_data, feature_names=features.columns, ax=ax[i], show=False)

# Remove any unused subplots
for j in range(11, len(ax)):
    fig.delaxes(ax[j])

plt.tight_layout()
plt.show()
#shap.dependence_plot(2, shap_values, test_data, feature_names=features.columns)
#shap.summary_plot(shap_values, test_data, feature_names=features.columns)

In [ ]:
shap.initjs()
shap.plots.force(explainer.expected_value[0], shap_values[..., 0], feature_names=features.columns)